In [63]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [64]:
directory = "data/mnist_images_csv/"
df = pd.read_csv(directory + "train.csv")
# df=pd.read_csv("data/mnist_images_csv/train.csv")

file_paths = df["file_name"].values
print(file_paths)   
labels = df["label"].values
print(labels)
ds_train = tf.data.Dataset.from_tensor_slices((file_paths, labels))

['0_1.jpg' '0_2.jpg' '0_3.jpg' '0_4.jpg' '0_5.jpg' '1_1.jpg' '1_2.jpg'
 '1_3.jpg' '1_4.jpg' '1_5.jpg' '2_1.jpg' '2_2.jpg' '2_3.jpg' '2_4.jpg'
 '2_5.jpg' '3_1.jpg' '3_2.jpg' '3_3.jpg' '3_4.jpg' '3_5.jpg' '4_1.jpg'
 '4_2.jpg' '4_3.jpg' '4_4.jpg' '4_5.jpg' '5_1.jpg' '5_2.jpg' '5_3.jpg'
 '5_4.jpg' '5_5.jpg' '6_1.jpg' '6_2.jpg' '6_3.jpg' '6_4.jpg' '6_5.jpg'
 '7_1.jpg' '7_2.jpg' '7_3.jpg' '7_4.jpg' '7_5.jpg' '8_1.jpg' '8_2.jpg'
 '8_3.jpg' '8_4.jpg' '8_5.jpg' '9_1.jpg' '9_2.jpg' '9_3.jpg' '9_4.jpg'
 '9_5.jpg']
[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 6 6 6 6 6 7 7
 7 7 7 8 8 8 8 8 9 9 9 9 9]


In [65]:
df.head(3)

,file_name,label
0,0_1.jpg,0
1,0_2.jpg,0
2,0_3.jpg,0


In [66]:
def read_image(image_file, label):
    # image = tf.io.read_file(directory + image_file)
    image = tf.io.read_file(directory + image_file)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
    return image, label

In [67]:
#checking the above function
image_file = "8_5.jpg"
label=8
# Call the read_image function
img,lbl = read_image(image_file, label)
print(img)
print(lbl)

tf.Tensor(
[[[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.01960784]
  [0.03137255]
  [0.        ]
  [0.        ]
  [0.00392157]
  [0.05490196]
  [0.        ]
  [0.02745098]
  [0.        ]
  [0.        ]
  [0.02352941]
  [0.        ]
  [0.        ]
  [0.03529412]
  [0.01568628]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.06666667]
  [0.01176471]
  [0.        ]
  [0.00392157]
  [0.03921569]
  [0.        ]
  [0.        ]
  [0.03921569]
  [0.        ]
  [0.03921569]
  [0.01960784]
  [0.01568628]
  [0.0509804 ]
  [0.04313726]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.02745

In [68]:
def augment(image, label):
    # data augmentation here
    return image, label

In [69]:
ds_train = ds_train.map(read_image).map(augment).batch(2)

In TensorFlow, when you chain multiple .map() operations together, each operation applies to the output of the previous one. In this case, ds_train.map(read_image) first maps the read_image function to each element of the dataset, and then ds_train.map(augment) maps the augment function to the results of the previous mapping. Finally, .batch(2) batches the elements in groups of 2.

In [70]:
# # Print and display some decoded images
# import matplotlib.pyplot as plt
# for image_batch, label_batch in ds_train.take(1):  # Take the first batch
#     for i in range(len(image_batch)):
#         image = image_batch[i].numpy()
#         label = label_batch[i].numpy()
        
#         # Display the image and label (you can modify this part)
#         plt.imshow(image.squeeze(), cmap='gray')
#         plt.title(f"Label: {label}")
#         plt.show()

In [71]:
# checking if the dataset is empty or not
dataset_size = tf.data.experimental.cardinality(ds_train).numpy()
if dataset_size == 0:
    print("The dataset is empty.")
else:
    print("The dataset contains", dataset_size, "elements.")

The dataset contains 25 elements.


In [72]:
print(ds_train)
print(ds_train.element_spec)


<_BatchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


In [73]:
# for epoch in range(10):
#     for x, y in ds_train:
#         print("Image shape:", x.shape)
#         print("Label:", y)

In [74]:
model = keras.Sequential(
    [
        layers.Input((28, 28, 1)),
        layers.Conv2D(16, 3, padding="same"),
        layers.Conv2D(32, 3, padding="same"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(10),
    ]
)

In [75]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=True),],
    metrics=["accuracy"],
)


In [76]:
model.fit(ds_train, epochs=10, verbose=2)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Input is empty.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_2672]